# Feature Extraction

In [2]:
import pandas as pd
import numpy as np

import multiprocessing as mp
from joblib import Parallel, delayed

from preprocessing_functions.py import mfcc_chroma_parser
from beat_features import beat_feat_parser, static_tempo # Importing from .py file to keep things more readable

**Testing the function**

In [ ]:
dnb = r"D:\Music\DnB\beatjunkierato+dnbpanacea1hitfreedl.mp3"
beat = beat_feat_parser(dnb)

## Beat features

The beat features is based on the ["Advanced Usage"](https://librosa.github.io/librosa/tutorial.html#advanced-usage) section in their tutorial.  This extracts features by "integrating harmonic-percussive separation, multiple spectral features, and beat-synchronous feature aggregation." I'll give a (mostly) high level overview of what these mean and why I chose to utilize them.  I'll also explain the behind-the-scenes action of the function because the actual code isn't that easy to understand without some prior knowledge.

In general, using raw features from an audio time series is not the best route to go.  There are many reasons for this (different scales, units, etc) but one example is a problem I encountered before I got this working.  The resultant output of the function gave a different NumPy array for each file.  One axis was constant: it was the index for the feature "names."  The pitch chroma thats computed has 12 components that correspond to the instensity of each pitch (by [pitch](https://en.wikipedia.org/wiki/Pitch_(music)) I mean the familiar C, D, E, F, G, A, B scale).  The other axis was the value of the coefficient at a frame, which is based on the length of the audio file.  Besides the fact that dealing with a 12x16000 tensor isn't particularly fun, the size difference also is an issue for comparing audio tracks.  

This can be handled by aggregating the features in a multitude of ways.  You can do this statistically with the mean, median, standard deviation, etc, or change how you extract the features, or some other clever way that gets you want you need.  Additionally, these aggregations actually can give you more info about the audio file because there could be meaning in the statistical features.  I chose to use the arithmetic mean because it was easy to implement and it would still give a meaningful representation of each song.

Now to look at what was actually aggregated and how I got them.

#### Harmonic-Percussive Source Separation (HPSS)

Without going too much into the math behind this, HPSS takes the Fourier transform of short windows (on the order of ms) that slightly overlap which then allows you to separate the harmonic and percussive elements of the song.  This allows easier extraction of harmonic features like pitch and percussive features like rhythm/tempo.  

#### Mel-frequency cepstral coefficients (MFCCs)

See [LibROSA EDA](./notebooks/Notebooks/EDA_LibROSA.ipynb)

#### MFCC Deltas

The smooth first order differences between the MFCC columns.  (Same as in time series analysis).

#### Beat MFCC Delta

The MFCCs and are aggregated by matching them to a time index (beat frames in this case).

#### Chromagram

Same thing as the [pitch chroma]() from the LibROSA EDA, although this one is computed with a slightly different method.  Explaining the differences would involve too much math for this but you can read more if you're curious: 
- [Short-time Fourier Transform](https://en.wikipedia.org/wiki/Short-time_Fourier_transform)  
- [Constant-Q transform](https://en.wikipedia.org/wiki/Constant-Q_transform)

#### Beat Chroma

Similar to the beat MFCC delta but with the chromagram.  This is also aggregated with the median instead of the mean.

---

The output is a single feature vector for the audio file to be used in the neural network!

In [ ]:
# np.savetxt('subset_beat.npy', beat_features, delimiter=',')

In [ ]:
features = np.loadtxt('./data/subset_beat.npy', delimiter=',')

In [ ]:
features.shape

**Getting the static tempo for each track**

I looped through each genre individually so I could set the proper initial bpm estimate.  The accuracy of the algorithm is highly dependent on the initial estimate.

In [ ]:
# tempos = np.empty((1,))
# for files in glob.glob("D:\MusicSubset\DnB\*.mp3"):
#     tempos = np.vstack((tempos,static_tempo(files, bpm_estimate=140)))
# tempos.shape

In [ ]:
# for files in glob.glob("D:\MusicSubset\Dubstep\*.mp3"):
#     tempos = np.vstack((tempos,static_tempo(files, bpm_estimate=150)))
# tempos.shape

In [ ]:
# for files in glob.glob("D:\MusicSubset\Garage\*.mp3"):
#     tempos = np.vstack((tempos,static_tempo(files, bpm_estimate=120)))
# tempos.shape

In [ ]:
# for files in glob.glob("D:\MusicSubset\House\*.mp3"):
#     tempos = np.vstack((tempos,static_tempo(files, bpm_estimate=120)))
# tempos.shape

In [ ]:
# for files in glob.glob("D:\MusicSubset\Indie\*.mp3"):
#     tempos = np.vstack((tempos,static_tempo(files, bpm_estimate=120)))
# tempos.shape

In [ ]:
# for files in glob.glob("D:\MusicSubset\Techno\*.mp3"):
#     tempos = np.vstack((tempos,static_tempo(files, bpm_estimate=150)))
# tempos.shape

In [ ]:
# np.savetxt("subset_tempos1.npy", tempos, delimiter=",")

Trying to speed up the process with pooling.  It wasn't very successful, which I will explain in the next notebook.

In [ ]:
pool = mp.Pool(mp.cpu_count())

In [ ]:
path = "D:\Music\DnB\*.mp3"

In [ ]:
# feats = np.empty((92,))
# feats = np.dstack(pool.map(beat_feat_parser, [file for file in glob.glob(path)]))
# pool.close()

In [ ]:
feats = pd.DataFrame(beat_feats)

In [ ]:
feats

In [ ]:
feats.to_csv('feats.csv', index=False)

In [ ]:
pd.read_csv('../data/track_df.csv').shape

### Extracting features on the whole data set

This step was done on the Google Cloud Platform's (GCP) Compute Engine (GCE).  The total size of the .mp3 files was about 13 GB and it the functions took way too long to run on my local machine.  After timing the subset I estimated it would have taken around 30 hrs to do it on my laptop.  However this was before I discovered the [`Multiprocessing`](https://timber.io/blog/multiprocessing-vs-multithreading-in-python-what-you-need-to-know/) and [`joblib`](https://joblib.readthedocs.io/en/latest/parallel.html) libraries.  Even then it wasn't feasible to lock up my computer for 4-6 hrs and I kept getting errors, or restarting even though I set it to not restart (thanks Windows automatic updater!),  so setting and leaving it was too risky.

My GCE instance had 24 cpu cores and 40 Gb of RAM (it still took 3-4 hrs plus many retries) and I ended up having to delete about 10 corrupted files.

The beat features were also an issue becuase they took too long to process and there was an issue with the tempo being inaccurate.  The final features came from the `mfcc_chroma_parser`, a combination of a few functions I described earlier, and the static tempo estimate.  Because the beat features extraction was such a failure, I ran through each genre separately and made sure I labeled them individually to ensure mismatching wasn't an issue.    

In [ ]:
# Path I passed into the glob.glob method to loop through the files.
# path = "./music/Music/*/*.mp3"    

In [ ]:
# Parallelizing the extraction

# feats = Parallel(n_jobs=24)(delayed(mfcc_chroma_parser)(file) for file in glob.glob(path))

In [ ]:
# Each genre was saved as both a .npy file then converted to a labeled dataframe 
# and saved again to a .csv file for extra safety and convenience.

# dnb_feats = np.asarray(feats)
# np.savetxt("dnb_feats.npy", dnb_feats, delimiter=",")
# dnb = pd.DataFrame(dnb_feats)
# dnb['label'] = 1
# dnb.to_csv('garage.csv', index=False)

# dstep_feats = np.asarray(feats)
# np.savetxt("dstep_feats.npy", dstep_feats, delimiter=",")
# dstep = pd.DataFrame(dstep_feats)
# dstep['label'] = 2
# dstep.to_csv('dstep.csv', index=False)

# garage_feats = np.asarray(feats)
# np.savetxt("garage_feats.npy", garage_feats, delimiter=",")
# garage = pd.DataFrame(garage_feats)
# garage['label'] = 3
# garage.to_csv("garage.csv", index=False)

# house_feats = np.asarray(feats, dtype='float32')
# np.savetxt("house_feats.npy", house_feats, delimiter=",")
# house = pd.DataFrame(house_feats)
# house.label = 4
# house.to_csv('house.csv', index=False)

# indie_feats = np.asarray(feats)
# np.savetxt("indie_feats.npy", indie_feats, delimiter=",")
# indie = pd.DataFrame(indie_feats)
# indie.label = 5
# indie.to_csv('indie.csv', index=False)

# techno_feats = np.asarray(feats)
# np.savetxt('techno_feats.csv', techno_feats, delimiter=False)
# techno = pd.DataFrame(techno_feats)
# techno.label = 6
# techno.to_csv('techno.csv', index=False)